# <center> Taller Keras $-$ Ejemplo 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

from tensorflow.keras.models import Model, Sequential, load_model, save_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

from pandas import DataFrame
from sklearn.metrics import confusion_matrix, classification_report
from plotmatrix import pretty_plot_confusion_matrix, PlotMatrix

import warnings
warnings.filterwarnings('ignore')

# Introducción a las Redes Neuronales (NNs)

__Objetivos:__
* Implementar una NN de varias capas en Keras para regresión.

In [ ]:
# Construimos datos sintéticos

nsample = 1000
sig = 0.5

x = np.linspace(-1, 1, nsample)
#X = np.column_stack((np.ones(nsample), x, (x-1)**2, np.sin(5*x), np.exp(x), np.exp(2.*x)))
#beta = [5., 0.5, -0.5, -10., -2.5, 3.]
X = np.column_stack((np.ones(nsample), x, (x-1)**2, np.sin(5*x)))
beta = [5., 3., -0.2, -10.]

y_true = np.dot(X, beta)
y = y_true + sig * np.random.normal(size=nsample)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(x, y, 'r', label='noisy data')
plt.plot(x, y_true, 'k--', label='true')
plt.legend()
plt.show()

### Split train-test

In [ ]:
p = 0.9
n = x.shape[0]
idx = np.random.permutation(n)

cut = int(p*n)

In [ ]:
x_train = X[idx[:cut], :]
x_test  = X[idx[cut:], :]
y_train = y_true[idx[:cut]]
y_test  = y_true[idx[cut:]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
print('Modified data shapes:')
print('X train:', x_train.shape, 'Y train:', y_train.shape)
print('X test:', x_test.shape, 'Y test:', y_test.shape)

data_shape = x_train.shape[1:]

print('')
print('data shape:', data_shape)

## Red neuronal multicapa

![](nn2.png)

__Ejercicio:__ Diseñe una red neuronal con 4 capas ocultas, más una capa de salida (como en la figura). Las capas deben ser densas. El número de neuronas en las capas debe ser: 512, 512, 256, 128, y 10 en la capa de salida.

Use funciones de activación ReLU (excepto en la capa de salida, donde se usará softmax). 


In [ ]:
### START CODE HERE ### (≈ 6 lines of code)
#network2 =

### END CODE HERE ###



def NNmulticapaReg(input_shape):
    I = Input(shape=input_shape, name='input')
    X = Dense(64, activation='relu', name='dense1')(I)
    X = Dense(32, activation='relu', name='dense2')(X)
    X = Dense(16, activation='relu', name='dense3')(X)
    X = Dense(1, activation=None, name='classifier')(X)
    model = Model(I, X, name='NN-Multicapa-Reg')
    return model

In [ ]:
if 'network' in globals(): del network
network = None

network = NNmulticapaReg(data_shape)

In [ ]:
network.summary()

__Ejercicio:__ Compile y entrene la red neuronal de 4 capas.

Utilice el optimizador 'RMSprop', usted elige el learning rate apropiado. Use la función de pérdida 'categorical_crossentropy', y use como métrica el 'accuracy'.

Para el entrenamiento, use 15 épocas (iteraciones), un tamaño de batch de 64 ó 128, y haga una partición de validación de 0.16666 (5/6).

### Compile

In [ ]:
alpha = 1e-3
opt = optimizers.Adam(lr=alpha, decay=1e-4)
network.compile(optimizer=opt, loss='mse', metrics=['mae'])

### Train

In [ ]:
tic = time()
history = network.fit(x_train, y_train, epochs=40, batch_size=128, validation_split=0.15)
toc = time()
print('total training time:', toc-tic, 'seconds')

### Ver el historial de desempeño de la red

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
mse = history.history['mae']
val_mse = history.history['val_mae']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(mse)+1)

# figure
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, mse, 'ro', label='Training MSE')
plt.plot(epochs, val_mse, 'r', label='Validation MSE')
plt.title('Training and validation MSE')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

### Resultados

In [ ]:
#results2 = network2.evaluate(test_images, test_labels)
#print('results loss:', results2[0])
#print('results accuracy:', results2[1])

In [ ]:
# compute predictions (inference process)
pred_train = network.predict(x_train)
pred_test  = network.predict(x_test)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(y_train, pred_train, 'b.', label='true')
plt.legend()
plt.show()

In [ ]:
minn = min(y_test.min(), pred_test.min())
maxx = min(y_test.max(), pred_test.max())

plt.figure(figsize=(5,5))
plt.plot([minn,maxx], [minn,maxx], 'k-')
plt.plot(y_test, pred_test, 'b.', label='true')
plt.legend()
plt.show()

Hacemos un plot de la gráfica real contra la predicción sobre el dominio

In [ ]:
#jdx = np.argsort(idx)
xx = x[idx[cut:]]
yy = y_true[idx[cut:]]
yyn = y[idx[cut:]]
pred = network.predict(X[idx[cut:]])

In [ ]:
jdx = np.argsort(xx)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(xx[jdx], yyn[jdx], 'b.', label='noisy data')
plt.plot(xx[jdx], yy[jdx], 'k--', label='true')
plt.plot(xx[jdx], pred[jdx], 'r', label='predicted')
plt.legend()
plt.show()

---

---